# Transferred Topics

In [1]:
import pandas as pd

def source_corpus(topic):
    return 'cw09' if (topic <= 200) else 'cw12'

def source_year(topic):
    if topic <= 50:
        return '2009'
    elif topic <= 100:
        return '2010'
    elif topic <= 150:
        return '2011'
    elif topic <= 200:
        return '2012'
    elif topic <= 250:
        return '2013'
    elif topic <= 300:
        return '2014'
    raise

def is_shallow_topic(topic):
    return topic['relevant'] > 0 and topic['irrelevant'] > 0 and (topic['relevant'] + topic['irrelevant']) >= 10
    
df = pd.read_json('../../test/resources/overview-of-transferred-topics.jsonl', lines=True)
df['sourceCorpus'] = df.topic.apply(lambda i: source_corpus(i))
df['sourceYear'] = df.topic.apply(lambda i: source_year(i))
df['shallowTopic'] = df.apply(lambda i: is_shallow_topic(i), axis=1)

In [3]:
def extract_shallow_topic_count(df, year: str, targetCorpus: str):
    df = df.groupby(['sourceYear', 'targetCorpus']).agg({
        'sourceCorpus': 'first',
        'topic': 'count',
        'shallowTopic': 'sum'
    })
    
    for k, v in df.iterrows():
        if (year, targetCorpus) == k:
            return str(int(v['shallowTopic']))
    
    raise

def table_row(df, year):
    if year in ['2013', '2014']:
        return '& ' + year + ' & -- & -- & ' + \
        extract_shallow_topic_count(df, year, 'cc15') + ' \\\\\n'
    
    return '& ' + year + ' & ' +\
        extract_shallow_topic_count(df, year, 'cw12') + ' & ' + \
        extract_shallow_topic_count(df, year, 'cw12wb12') + ' & ' + \
        extract_shallow_topic_count(df, year, 'cc15') + ' \\\\\n'
    
def create_table_transferred_shallow_judgments(df):
    df = df.groupby(['sourceYear', 'targetCorpus']).agg({
        'sourceCorpus': 'first',
        'topic': 'count',
        'shallowTopic': 'sum'
    })
    
    return """
\\begin{table}[tb]
\\caption{TODO: Add some plots from the other jupyter notebook (transferred judgments overall, ratio ov transferred relevant vs irrelevant documents etc.)}

\\label{table-transferred-shallow-judgments}
\\small
\\centering
\\setlength{\\tabcolsep}{12pt}
\\begin{tabular}{@{}c@{}c@{\\hspace{1em}}c@{\\ \\ }c@{\\ \\ }c@{}}
\\toprule
\\multicolumn{2}{@{}l@{}}{\\bfseries Track} & \\multicolumn{3}{@{}c@{\\hspace{1em}}}{\\bfseries Shallow Topics}\\\\
\\cmidrule{3-5}
&& CW12 & CW12$^{\\dagger}$ & CC15\\\\
\\midrule
\\addlinespace
\\parbox[t]{4mm}{\\multirow{4}{*}{\\rotatebox[origin=c]{90}{\\tiny CW09}}} """ + \
table_row(df, '2009') + \
table_row(df, '2010') + \
table_row(df, '2011') + \
table_row(df, '2012') + \
"""
\\midrule
\\parbox[t]{4mm}{\\multirow{2}{*}{\\rotatebox[origin=c]{90}{\\tiny CW12}}} """ + \
table_row(df, '2013') + \
table_row(df, '2014') + \
"""
\\bottomrule
\\end{tabular}
\\end{table}
"""

table_transferred_shallow_judgments = create_table_transferred_shallow_judgments(df)
with open('/sigir21/sigir21-relevance-label-transfer-paper-submitted/table-transferred-shallow-judgments.tex', 'w+') as f:
    f.write(table_transferred_shallow_judgments)